In [0]:
import keras
import pandas as pd
import csv
import os
from sklearn import linear_model
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from joblib import dump, load

In [0]:
dir_path = '/content/drive/My Drive/NASA - Integrated Assessment/cov data/split/'
header = ['iso_code','location','date','total_cases','new_cases','total_deaths','new_deaths','total_cases_per_million','new_cases_per_million','total_deaths_per_million','new_deaths_per_million','total_tests','new_tests','total_tests_per_thousand','new_tests_per_thousand','new_tests_smoothed','new_tests_smoothed_per_thousand','tests_units','stringency_index','population','population_density','median_age','aged_65_older','aged_70_older','gdp_per_capita','extreme_poverty','cvd_death_rate','diabetes_prevalence','female_smokers','male_smokers','handwashing_facilities','hospital_beds_per_100k']

In [0]:
drop_vs = ['iso_code', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths', 'total_tests','new_tests','new_tests_smoothed','new_tests_smoothed_per_thousand','tests_units','median_age','aged_70_older','cvd_death_rate','diabetes_prevalence','female_smokers','male_smokers','handwashing_facilities','extreme_poverty']
single_vs = ['population_density','aged_65_older','gdp_per_capita','hospital_beds_per_100k']
save_vs = ['location','population']
check = ['total_cases_per_million','new_cases_per_million', 'total_deaths_per_million','new_deaths_per_million','total_tests_per_thousand','new_tests_per_thousand','stringency_index']  

In [0]:
gini_access = {}
gf = open('/content/drive/My Drive/NASA - Integrated Assessment/cov data/combined_gini_access.csv', "r")
reader = csv.reader(gf, delimiter=',')
for ds in reader:
  gini_access[ds[0].lower()] = [ds[1],ds[2]]
defaults = {'population_density': 14.7, 'aged_65_older': 20, 'gdp_per_capita': 18381, 'hospital_beds_per_100k': 0, 'gini': 38, 'access': 442}

In [0]:
X = []
tcpm = []
ncpm = []
tdpm = []
ndpm = []
ttpt = []
ntpt = []
stidx = []

In [6]:
for filename in os.listdir(dir_path):
  country_data = pd.read_csv(dir_path + filename, header = None)
  country_data.columns = header

  country_data = country_data.fillna(0)
  single_use = {}
  for s in single_vs:
    single_use[s] = country_data[s][0]

  save_data = {}
  for s in save_vs:
    save_data[s] = country_data[s][0]

  drop = []
  drop.extend(drop_vs)
  drop.extend(single_vs)
  drop.extend(save_vs)

  country_data = country_data.drop(columns=drop)

  n_samples = len(country_data.index)-1
  for k in range(n_samples):
    ap = []
    for dat in check:
      inpd = country_data[dat][k]
      if inpd == 0:
        inpd = 0.00001
      ap.append(inpd)
    for dat in single_vs:
      if single_use[dat] != 0:
        ap.append(single_use[dat])
      else:
        ap.append(defaults[dat])
    ga = []
    try:
      ga = [float(gini_access[save_data['location'].lower()][0]), float(gini_access[save_data['location'].lower()][1])]
    except:
      ga = [float(defaults['gini']), float(defaults['access'])]
    ap.extend(ga)
    X.append(ap)

  for k in range(1,n_samples+1):
    tcpm.append(country_data['total_cases_per_million'][k])
    ncpm.append(country_data['new_cases_per_million'][k])
    tdpm.append(country_data['total_deaths_per_million'][k])
    ndpm.append(country_data['new_deaths_per_million'][k])
    ttpt.append(country_data['total_tests_per_thousand'][k])
    ntpt.append(country_data['new_tests_per_thousand'][k])
    stidx.append(country_data['stringency_index'][k])
  print(save_data['location'])

Aruba
Afghanistan
Angola
Anguilla
Albania
Andorra
United Arab Emirates
Argentina
Armenia
Antigua and Barbuda
Australia
Austria
Azerbaijan
Burundi
Belgium
Benin
Bonaire Sint Eustatius and Saba
Burkina Faso
Bangladesh
Bulgaria
Bahrain
Bahamas
Bosnia and Herzegovina
Belarus
Belize
Bolivia
Brazil
Barbados
Brunei
Bhutan
Botswana
Central African Republic
Canada
Switzerland
Chile
China
Cote d'Ivoire
Cameroon
Democratic Republic of Congo
Congo
Colombia
Comoros
Cape Verde
Costa Rica
Cuba
Curacao
Cayman Islands
Cyprus
Czech Republic
Germany
Djibouti
Dominica
Denmark
Dominican Republic
Algeria
Ecuador
Egypt
Eritrea
Western Sahara
Spain
Estonia
Ethiopia
Finland
Fiji
Falkland Islands
France
Faeroe Islands
Gabon
United Kingdom
Georgia
Guernsey
Ghana
Gibraltar
Guinea
Gambia
Guinea-Bissau
Equatorial Guinea
Greece
Grenada
Greenland
Guatemala
Guam
Guyana
Hong Kong
Honduras
Croatia
Haiti
Hungary
Indonesia
Isle of Man
India
Ireland
Iran
Iraq
Iceland
Israel
Italy
Jamaica
Jersey
Jordan
Japan
Kazakhstan
Keny

In [10]:
reg_tcpm = linear_model.BayesianRidge() 
reg_ncpm = linear_model.BayesianRidge() 
reg_tdpm = linear_model.BayesianRidge()
reg_ndpm = linear_model.BayesianRidge() 
reg_ttpt = linear_model.BayesianRidge() 
reg_ntpt = linear_model.BayesianRidge() 
reg_stidx = linear_model.BayesianRidge() 

reg_tcpm.fit(X,tcpm)
reg_ncpm.fit(X,ncpm)
reg_tdpm.fit(X,tdpm)
reg_ndpm.fit(X,ndpm)
reg_ttpt.fit(X,ttpt)
reg_ntpt.fit(X,ntpt)
reg_stidx.fit(X,stidx)

save_path = '/content/drive/My Drive/NASA - Integrated Assessment/cov data/bayesian regression models/'
dump(reg_tcpm, save_path + "tcpm.joblib")
dump(reg_ncpm, save_path + "ncpm.joblib")
dump(reg_tdpm, save_path + "tdpm.joblib")
dump(reg_ndpm, save_path + "ndpm.joblib")
dump(reg_ttpt, save_path + "ttpt.joblib")
dump(reg_ntpt, save_path + "ntpt.joblib")
dump(reg_stidx, save_path + "stidx.joblib")

['/content/drive/My Drive/NASA - Integrated Assessment/cov data/bayesian regression models/stidx.joblib']

In [8]:
'''
X = []  
for k in range(n_samples,n_samples+1):
  ap = []
  for dat in check:
    inpd = country_data[dat][k]
    ap.append(inpd)
  for dat in sud:
    if single_use[dat] != 0:
      ap.append(single_use[dat])
    else:
      ap.append(default[dat])
  ga = []
  try:
    ga = [float(gini_access[save_data['location'].lower()][0]), float(gini_access[save_data['location'].lower()][1])]
  except:
    ga = [float(default['gini']), float(default['access'])]
  ap.extend(ga)
  X.append(ap)
print(X)
'''
x = [[0,0,0,0,0,0,0,0,0,0,0,0,0]]
reg.predict(x)

array([-1.92192344])

In [9]:
reg.coef_

array([ 1.25816581e-02,  2.08440595e-01, -7.06310368e-02,  1.60516597e+00,
       -5.09604786e-01,  2.20607035e+01,  6.01874540e-02,  1.78305552e-04,
       -1.88581162e-01,  2.69699609e-04, -3.38033210e-01,  5.83381073e-03,
       -2.74617727e-04])

In [13]:
#https://scikit-learn.org/stable/modules/model_persistence.html
dump(reg, '/content/drive/My Drive/NASA - Integrated Assessment/cov data/MODEL.joblib') 

['/content/drive/My Drive/NASA - Integrated Assessment/cov data/MODEL.joblib']